In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [70]:
from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression

def loss(beta, X, y, lam, p):
    # Logistic loss with L_p regularization
    logistic_loss = np.mean(np.log(1 + np.exp(-y * np.dot(X, beta))))
    reg_loss = lam * np.linalg.norm(beta, p)
    return logistic_loss + reg_loss

def gradient(beta, X, y, lam, p):
    # Gradient of logistic loss
    logistic_grad = np.dot(X.T, -y / (1 + np.exp(y * np.dot(X, beta))))
    # Gradient of regularization term
    if p == 1:
        reg_grad = lam * np.sign(beta)
    else:
        norm_beta_p_minus_1 = np.power(np.abs(beta), p - 1)
        reg_grad = lam * np.multiply(np.sign(beta), norm_beta_p_minus_1)
        reg_grad /= np.power(np.linalg.norm(beta, p), p-1)
    
    return logistic_grad + reg_grad

def logistic_regression(X, y, lam, p):
    # Initialize beta with zeros or logistic regression coefficients as a starting point
    beta_init = LogisticRegression().fit(X, y).coef_[0]   
    # Define the objective function (loss) to minimize
    obj_func = lambda beta: loss(beta, X, y, lam, p)
    # Define the gradient of the objective function
    obj_grad = lambda beta: gradient(beta, X, y, lam, p)
    # Use scipy.optimize.minimize with the 'BFGS' method, providing the gradient
    result = minimize(fun=obj_func, x0=beta_init, jac=obj_grad, method='BFGS')
    return result.x

def predict(X, y, beta):
    # y_one is the probability of class 1
    y_one = np.exp(np.dot(X, beta)) / (1 + np.exp(np.dot(X, beta)))
    # y_pred is the predicted class
    y_pred = np.sign(y_one - 0.5)

    # check the correct accuracy 
    accuracy = np.mean(y == y_pred)
    return accuracy
    


In [71]:
def generate_logistic_regression_data(n_samples, n_features, beta_true, var, pct):
    # generate logistic regression data
    # var is the variance of the noise added to X
    # pct is the percentage of the Y that will be flipped

    # generate data 
    X = np.random.randn(n_samples, n_features)
    X = np.hstack((np.ones((n_samples, 1)), X))  
    logits = np.dot(X, beta_true)
    probabilities = 1 / (1 + np.exp(-logits))
    y = np.where(probabilities >= 0.5, 1, -1)

    # add noise to the data
    X = X + np.random.normal(0, var, X.shape)
    for i in range(int(n_samples * pct)):
        if y[i] == 1:
            y[i] = -1
        else:
            y[i] = 1
    
    return X, y  

def optimal_radius(X, q):
    A = np.dot(X.T, X) / X.shape[0]
    Z = np.random.multivariate_normal(np.zeros(X.shape[1]), A, 1000)
    norm_Z = np.linalg.norm(Z, ord=q, axis=1)
    eta = np.percentile(norm_Z, 95)
    radius = eta / np.sqrt(X.shape[0])
    return radius

In [72]:
# generate data with different noise
n_samples = 1000
n_features = 10
beta_true = np.random.randn(n_features + 1)

var_values = [0.5, 1, 2]
pct_values = [0.1, 0.2, 0.3]

# generate data with different noise
X_data = {}
y_data = {}
for var in var_values:
    X_data[var] = {}
    y_data[var] = {}
    for pct in pct_values:
        X, y = generate_logistic_regression_data(n_samples, n_features, beta_true, var, pct)
        X_data[var][pct] = X
        y_data[var][pct] = y


In [79]:
from sklearn.model_selection import train_test_split

p_values = [1, 2, 3, 4, 5]
p_conjugate = []
for p in p_values:
    if p == 1:
        p_conjugate.append(np.inf)
    else:
        p_conjugate.append(p / (p - 1))

# first, we need to find the optimal radius for each dataset
radius_values = {}
for var in var_values:
    radius_values[var] = {}
    for pct in pct_values:
        X = X_data[var][pct]
        radius_values[var][pct] = {}
        for idx in range(len(p_values)):
            radius_values[var][pct][p_values[idx]] = optimal_radius(X, p_conjugate[idx])

# now, for each var, pct, and p, we will calculate the accuracy 
accuracy_values = {}
for var in var_values:
    accuracy_values[var] = {}
    for pct in pct_values:
        X = X_data[var][pct]
        y = y_data[var][pct]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
        accuracy_values[var][pct] = {}
        for idx in range(len(p_values)):
            p = p_values[idx]
            radius = radius_values[var][pct][p]
            beta = logistic_regression(X_train, y_train, radius, p)
            accuracy = predict(X_test, y_test, beta)
            accuracy_values[var][pct][p] = accuracy

        # logistic regression withou Lp regularization
        beta = logistic_regression(X_train, y_train, 0, 1)
        accuracy = predict(X_test, y_test, beta)
        accuracy_values[var][pct][0] = accuracy

accuracy_values



{0.5: {0.1: {1: 0.8075, 2: 0.8075, 3: 0.8075, 4: 0.8075, 5: 0.8075, 0: 0.8075},
  0.2: {1: 0.675, 2: 0.675, 3: 0.6725, 4: 0.675, 5: 0.675, 0: 0.6875},
  0.3: {1: 0.645, 2: 0.6275, 3: 0.6425, 4: 0.6425, 5: 0.6425, 0: 0.6375}},
 1: {0.1: {1: 0.7425, 2: 0.7425, 3: 0.7425, 4: 0.7425, 5: 0.7425, 0: 0.7425},
  0.2: {1: 0.5725, 2: 0.5925, 3: 0.5925, 4: 0.5925, 5: 0.5925, 0: 0.6025},
  0.3: {1: 0.575, 2: 0.5725, 3: 0.575, 4: 0.575, 5: 0.575, 0: 0.5725}},
 2: {0.1: {1: 0.595, 2: 0.585, 3: 0.585, 4: 0.585, 5: 0.5925, 0: 0.59},
  0.2: {1: 0.545, 2: 0.5475, 3: 0.55, 4: 0.55, 5: 0.55, 0: 0.5475},
  0.3: {1: 0.545, 2: 0.545, 3: 0.545, 4: 0.545, 5: 0.5475, 0: 0.545}}}

In [74]:
# accuracy_values get the accuracy for (var, pct) as column, and p as row
# print out the accuracy values tables as a dataframe
# let (var, pct) as column
# p as row 

data = []
for var, pct_dict in accuracy_values.items():
    for pct, p_dict in pct_dict.items():
        for p, accuracy in p_dict.items():
            data.append({'var': var, 'pct': pct, 'p': p, 'accuracy': accuracy})

df = pd.DataFrame(data)

# group the (var, pct) the couple as column
# p as row
df = df.pivot(index='p', columns=('var', 'pct'), values='accuracy')
df_latex = df.to_latex()
print(df_latex) 


\begin{tabular}{lrrrrrrrrr}
\toprule
var & \multicolumn{3}{l}{0.5} & \multicolumn{3}{l}{1.0} & \multicolumn{3}{l}{2.0} \\
pct &     0.1 &     0.2 &     0.3 &     0.1 &     0.2 &     0.3 &     0.1 &     0.2 &     0.3 \\
p &         &         &         &         &         &         &         &         &         \\
\midrule
0 &  0.7850 &  0.7225 &  0.6525 &  0.7000 &  0.6200 &  0.5550 &  0.5975 &  0.5700 &  0.5375 \\
1 &  0.7875 &  0.6900 &  0.6475 &  0.6975 &  0.6200 &  0.5600 &  0.5925 &  0.5700 &  0.5375 \\
2 &  0.7875 &  0.7150 &  0.6500 &  0.7050 &  0.6200 &  0.5575 &  0.5975 &  0.5700 &  0.5375 \\
3 &  0.7875 &  0.7175 &  0.6475 &  0.7025 &  0.6225 &  0.5600 &  0.6000 &  0.5675 &  0.5375 \\
4 &  0.7875 &  0.7175 &  0.6475 &  0.7050 &  0.6225 &  0.5600 &  0.6000 &  0.5650 &  0.5375 \\
5 &  0.7875 &  0.7175 &  0.6475 &  0.7150 &  0.6225 &  0.5600 &  0.6000 &  0.5675 &  0.5375 \\
\bottomrule
\end{tabular}



C:\Users\specf\AppData\Local\Temp\ipykernel_21320\938548336.py:17: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_latex = df.to_latex()


In [93]:
def cross_validation(X, y, p_values, p_conjugate):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    optimal_rad = {}
    cross_validation_rad = {}
    for idx in range(len(p_values)):
        p = p_values[idx]
        radius = optimal_radius(X_train, p)
        beta = logistic_regression(X_train, y_train, radius, p)
        accuracy = predict(X_test, y_test, beta)
        optimal_rad[p_values[idx]] = accuracy
        # use the cross validation to get the optimal radius

        radius_candidate = np.linspace(0.1, 1, 20)
        accuracy_values = []
        for radius in radius_candidate:
            beta = logistic_regression(X_train, y_train, radius, p)
            accuracy = predict(X_test, y_test, beta)
            accuracy_values.append(accuracy)

        cross_validation_radius = radius_candidate[np.argmax(accuracy_values)]
        cross_validation_rad[p_values[idx]] = cross_validation_radius

    return optimal_rad, cross_validation_rad


# generate a dataframe

a, b = cross_validation(X, y, p_values, p_conjugate)
print(a)
print(b)

cross_data = []
for var in var_values:
    for pct in pct_values:
        X = X_data[var][pct]
        y = y_data[var][pct]
        optimal_rad, cross_validation_rad = cross_validation(X, y, p_values, p_conjugate)
        for p in p_values:
            cross_data.append({'var': var, 'pct': pct, 'p': p, 'optimal_rad': optimal_rad[p]})

cross_df = pd.DataFrame(cross_data)
cross_df = cross_df.pivot(index='p', columns=('var', 'pct'), values=['optimal_rad'])
cross_df_latex = cross_df.to_latex()
print(cross_df_latex)


{1: 0.5725, 2: 0.575, 3: 0.5725, 4: 0.5725, 5: 0.5725}
{1: 0.19473684210526315, 2: 0.4789473684210527, 3: 0.7157894736842105, 4: 0.9526315789473684, 5: 1.0}
\begin{tabular}{lrrrrrrrrr}
\toprule
{} & \multicolumn{9}{l}{optimal\_rad} \\
var & \multicolumn{3}{l}{0.5} & \multicolumn{3}{l}{1.0} & \multicolumn{3}{l}{2.0} \\
pct &         0.1 &     0.2 &     0.3 &     0.1 &     0.2 &     0.3 &     0.1 &     0.2 &     0.3 \\
p &             &         &         &         &         &         &         &         &         \\
\midrule
1 &      0.7650 &  0.6675 &  0.5975 &  0.6675 &  0.5950 &  0.5850 &  0.6050 &  0.5925 &  0.5625 \\
2 &      0.7775 &  0.6875 &  0.5900 &  0.6825 &  0.5825 &  0.5825 &  0.6000 &  0.5950 &  0.5650 \\
3 &      0.7925 &  0.6875 &  0.5900 &  0.6850 &  0.5775 &  0.5825 &  0.5975 &  0.5950 &  0.5650 \\
4 &      0.7925 &  0.6875 &  0.5900 &  0.6900 &  0.5775 &  0.5850 &  0.5950 &  0.5950 &  0.5650 \\
5 &      0.7925 &  0.6875 &  0.6125 &  0.6900 &  0.5775 &  0.5800 &  0.5925

C:\Users\specf\AppData\Local\Temp\ipykernel_21320\1878901505.py:43: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  cross_df_latex = cross_df.to_latex()


In [94]:
cross_df

optimal_rad                                                          \
var         0.5                     1.0                     2.0           
pct         0.1     0.2     0.3     0.1     0.2     0.3     0.1     0.2   
p                                                                         
1        0.7650  0.6675  0.5975  0.6675  0.5950  0.5850  0.6050  0.5925   
2        0.7775  0.6875  0.5900  0.6825  0.5825  0.5825  0.6000  0.5950   
3        0.7925  0.6875  0.5900  0.6850  0.5775  0.5825  0.5975  0.5950   
4        0.7925  0.6875  0.5900  0.6900  0.5775  0.5850  0.5950  0.5950   
5        0.7925  0.6875  0.6125  0.6900  0.5775  0.5800  0.5925  0.5950   

             
var          
pct     0.3  
p            
1    0.5625  
2    0.5650  
3    0.5650  
4    0.5650  
5    0.5650